# 史実を辿る「走れメロス」計算 ♯1 ('24.05.24)

太宰治「走れメロス」では、主人公メロスは、友人セリヌンティウスを人質として残し、シラクスの市街から十里(40km)離れた村まで3日間かけて往復する。
18世紀ドイツで、フリードリヒ・フォン・シラーが書いた書物の日本訳※にもとづいて太宰が書いた、「走れメロス」物語にもとづいて、メロスの移動速度を検証した[優れた文献](https://www.rimse.or.jp/research/past/pdf/1st/work03.pdf)はあるが、さらに史実に遡った移動速度検証は無い。
そこで、ここではそうした計算を試みていく。

※ 新編シラー詩抄 小栗孝則訳 改造文庫（昭和12年）


「走れメロス」は、そのルーツを辿ると、紀元前400年頃のイタリア、ディオニュシオス1世・2世の時代の話だ。
さまざまなバリエーションがあるが、比較的多いパターンは、ディオニュシオスから死刑を宣告されたピュティアスが友人ダモンを人質にして家まで往復をする、というものだ。
往復するタイムリミットは、当日中というものもあれば、三日後や数日後のものもある。

%<img style="float:center;transform: rotate(0deg); height:10cm" src="./images/day_240524_geocoder_meros_Damon_and_Pythias.gif" />

```{figure} ./images/day_240524_geocoder_meros_Damon_and_Pythias.gif
---
height: 10cm
---
The story of Greeksの挿絵
```

## 竹馬の友セリヌンティウスと過ごした村には戻れない

まずは、太宰治版の物語に登場する名前にもとづくと、色々矛盾が出てくる。
メロスがシラクサ(Siracusa)の街に人質として残したのは、竹馬の友セリヌンティウスだ。
つまり、小さい頃から共に過ごした友人である。

セリヌンティウスというのは、「セリヌスの人」という名前なので、メロスとセリヌンティウスが暮らしていた場所は、
はシチリア島の南西部、セリヌス（Selinous）だろうか。
だとすると、メロスはSiracusaからSelinunteまでを三日間で往復したことになる。

SiracusaとSelinunteの緯度経度を調べ、地図に描いてみるとこうなる。

In [ ]:
# Geocoderパッケージを使う
import geocoder  # conda install conda-forge::geocoder
import time

# 地名文字列
place1='Siracusa,Italy'
place2='Selinunte,Italy'

# 緯度経度を取得する
siracusa=geocoder.osm(place1, timeout=5.0).latlng
time.sleep(3)
selinunte=geocoder.osm(place2, timeout=5.0).latlng
# 確認表示
print(place1, siracusa)
print(place2, selinunte)

In [14]:
# foliumパッケージを使う
import folium

#lat1,lon1 = siracusa.latlng
#lat2,lon2 = selinunte.latlng
lat1,lon1 = siracusa
lat2,lon2 = selinunte

# 地図を作る
fmap = folium.Map(location=siracusa,zoom_start=6)
#fmap = folium.Map(location=siracusa.latlng,zoom_start=6)

# マーカープロット
folium.Marker(location=[lat1,lon1],
              icon=folium.Icon(color='black')
             ).add_to(fmap)
folium.Marker(location=[lat2,lon2],icon=folium.Icon(color='blue')
             ).add_to(fmap)
fmap # 表示する

%<img style="float:center;transform: rotate(0deg); height:10cm" src="./images/day_240524_geocoder_meros_Damon_and_Pythias_map.png" />

```{figure} ./images/day_240524_geocoder_meros_Damon_and_Pythias_map.png
---
height: 10cm
---
SiracusaとSelinunteの位置
```

SiracusaとSelinunteは、シチリア島の中でもほとんど東西の反対側に位置していて、かなり遠い。
実際、距離を計算してみると、直線距離でも230kmほどもある。

In [15]:
# geopyパッケージを使う
from geopy.distance import geodesic # conda install conda-forge::geopy

distance_km = geodesic(siracusa, selinunte).km
print(distance_km,'(km)')

225.99443613592337 (km)


日暮れが遅い時期、たとえば夏至の頃のシチリア島での「日暮れ」を調べてみると、18時30分くらい。
すると、たとえば当日の日暮れまでの往復だと、直線距離でも時速53kmという非現実的な速度でなければ、メロスは往復できない。
タイムリミットが三日後の日暮れまでであれば、夜も昼も走り続けること前提で、時速5.6kmとなる。

In [17]:
# PyEphem astronomy libraryを使う
import ephem # conda install anaconda::ephem
import datetime

siracusa_obs = ephem.Observer()
siracusa_obs.lon = str(siracusa[0])
siracusa_obs.lat = str(siracusa[1])
siracusa_obs.date = ephem.Date('2024/06/21 13:0:0.0')
sun = ephem.Sun()

print('日の出時間：', siracusa_obs.next_rising(sun))
print('日暮れ時間：', siracusa_obs.next_setting(sun))

import math

h=siracusa[0]/15.
m=(h-math.floor(h))*60
h=math.floor(h)

print('＋',h,'時間',int(m),'分')

print('当日日暮れまでだった場合',226*2/8.5,'km/h')
print('三日後の日暮れだった場合',226*2/(24*3+8.5),'km/h')

日の出時間： 2024/6/22 03:02:32
日暮れ時間： 2024/6/21 16:04:53
＋ 2 時間 28 分
当日日暮れまでだった場合 53.1764705882353 km/h
三日後の日暮れだった場合 5.614906832298137 km/h


文献を辿っていくと、メロス（多くの文献ではピュティアス）が友人ダモンを人質として往復したのは、太宰が書いた十里先の村どころか、Siracusa市内のようだ。
それであれば、当日の日暮れまでというタイムリミットでも不思議ではなさそうだ。

## 文献を調べて計算をする検証の旅は続く

というわけで、「走れメロス」の検証の旅は、まだ始まったばかり。
過去の文献を調べ、さまざまな計算をしていけば、いつかきっと何か発見することができるかも？